In [1]:
## 4 functions
# get_advisory(list_input) : Download Advisory zip file and unzip to get shapefiles
# gdf = read_5dayForecast(list_input): Output 5-day forecast (center location, wind speed, etc)
# get_forwardspeed(list_input): Output forward movement direction and forward speed from text advisory
# get_max_wind(gdf): Output max wind speed from 5day hurricane forecast
# get_inland(gdf): Check if 5day hurricane location is inland or not

In [2]:
# Credit: Brent Austgen - https://github.com/infra-resil/bga-work/blob/master/notebooks/download-hand-0-2-1.ipynb
import os
import re
import urllib
from multiprocessing.pool import Pool
import requests
import zipfile
import pandas as pd
import geopandas as gpd
import urllib.request
from bs4 import BeautifulSoup

In [3]:
class Hurricane_Advisory:
    def __init__(self, _id, _yr, _adv_type, _adv_num):
        self.id = _id
        self.yr = _yr
        self.type = _adv_type
        self.num = _adv_num
        self.tide = '2'
        
    def get_storm(self):
        output = '%s%s%si%s.shp' % (self.direction, str(self.category), str(self.fws), str(self.tide))
        return output
        
    def get_advisory(self, working_dir): # Get hurricane advisory zip file from NOAA archive
        hurr_id = self.id
        hurr_yr = self.yr
        hurr_adv_type = self.type
        hurr_adv_num = self.num

        if len(str(hurr_adv_num)) == 1:
            hurr_adv_num = '00' + str(hurr_adv_num)
        elif len(str(hurr_adv_num)) == 2:
            hurr_adv_num = '0' + str(hurr_adv_num)
        suffix = '%s%s_%s_%s' % (hurr_id, hurr_yr, hurr_adv_type, hurr_adv_num)
        suffix_file = '%s.zip' % (suffix)

        url = 'https://www.nhc.noaa.gov/gis/forecast/archive/'
        advisory_url = url + suffix_file

        # setup directory for storing data
        data_dir = working_dir + str(hurr_yr) + '/' + hurr_id + '/' 
        if not os.path.exists(data_dir):
            os.makedirs(data_dir)

        # download advisory file
        r = requests.get(advisory_url)  
        save_location = data_dir + suffix_file
        with open(save_location, 'wb') as f:
            f.write(r.content)

        # unzip file
        zip_file = suffix_file
        zip_dir = suffix + '/'
        with zipfile.ZipFile(data_dir + zip_file,"r") as zip_ref:
            zip_ref.extractall(data_dir + zip_dir)

    
    def read_5dayForecast(self, working_dir):

        hurr_id = self.id
        hurr_yr = self.yr
        hurr_adv_type = self.type
        hurr_adv_num = self.num

        if len(str(hurr_adv_num)) ==1:
            hurr_adv_num = '00' + str(hurr_adv_num)
        elif len(str(hurr_adv_num)) == 2:
            hurr_adv_num = '0' + str(hurr_adv_num)

        # read point data
        shp_file_name = '%s%s-%s_%s_pts' % (hurr_id, hurr_yr, hurr_adv_num, hurr_adv_type)
        shp_file = shp_file_name + '.shp'

        data_dir = working_dir + str(hurr_yr) + '/' + hurr_id + '/' 
        suffix = '%s%s_%s_%s' % (hurr_id, hurr_yr, hurr_adv_type, hurr_adv_num)
        shp_file_directory = suffix + '/'

        # shp file location
        output_gdf = gpd.read_file(data_dir + shp_file_directory + shp_file)

        return output_gdf
    
    def get_max_wind(self, input_gdf):
        dict_ws = dict(zip(input_gdf['TAU'], input_gdf['MAXWIND']))

        max_ws = max(dict_ws.values()) # unit: knots

        max_ws_keys = []
        max_ws_keys_time = []
        for key, value in dict_ws.items():
            if value == max_ws:
                max_ws_keys.append(key)
                max_ws_keys_time.append(input_gdf.loc[input_gdf['TAU'] == key]['VALIDTIME'].values[0])

        if max_ws < 64:
            category = 0
        elif (max_ws >= 64) and (max_ws < 83):
            category = 1
        elif (max_ws >= 83) and (max_ws < 96):
            category = 2
        elif (max_ws >= 96) and (max_ws < 113):
            category = 3
        elif (max_ws >= 113) and (max_ws < 137):
            category = 4
        elif max_ws >= 137: 
            category = 5

        print('Max Wind(kt): %s, Category: %s, key: %s, Max Wind Hour: %s' % (max_ws, category, max_ws_keys, max_ws_keys_time))
        
        self.category = category
        
        return [max_ws, category, max_ws_keys, max_ws_keys_time]
    
    def get_inland(self, input_gdf):
        # Get Texas shapefile
        file_directory = '/Users/kyoung/Box Sync/qgis/gis/formatted/shp/'
        file_name = 'texas.shp'
        gdf_tx = gpd.read_file(file_directory + file_name)
        tx_poly = gdf_tx['geometry'].values[0]

        dict_center = dict(zip(input_gdf.TAU, input_gdf.geometry))

        inland_hrs = []
        for key, value in dict_center.items():
            if value.within(tx_poly):
                inland_hrs.append(key)
        min_hr = min(inland_hrs)

        min_hr_time = input_gdf.loc[input_gdf['TAU'] == min_hr]['VALIDTIME'].values[0]
        
        return [min_hr, min_hr_time]
    
    def get_forwardspeed(self):

        hurr_id = self.id
        hurr_yr = self.yr
        hurr_adv_type = self.type
        hurr_adv_num = self.num

        if len(str(hurr_adv_num)) ==1:
            hurr_adv_num = '00' + str(hurr_adv_num)
        elif len(str(hurr_adv_num)) == 2:
            hurr_adv_num = '0' + str(hurr_adv_num)

        url = 'https://www.nhc.noaa.gov/archive/%s/%s/%s%s.fstadv.%s.shtml?' % (hurr_yr, hurr_id, hurr_id, hurr_yr, hurr_adv_num)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        text_adv = soup.findAll('pre')[0]

        a = str(text_adv)
        b = a.split('\n')
        for i in b:
            if 'PRESENT MOVEMENT' in i:
                row = i

        forward_speed_string = row.split(' ')

        for i in range(len(forward_speed_string)):
            if forward_speed_string[i] == 'OR':
                position_dir = i
            elif forward_speed_string[i] == 'KT':
                position_fws = i

        fws = forward_speed_string[position_fws-1]
        fws_int = int(fws) * 1.15078
        print(fws_int)
        
        check_speed = [5, 10, 15]
        diff = 100

        for j in check_speed:
            temp_diff = abs(j - fws_int)
            
            if temp_diff < diff:
                diff = temp_diff
                fws_check = j    
        
        direction = forward_speed_string[position_dir-1]

        dict_direction = {'NORTH': 'n',
                          'NORTH-NORTHEAST' : 'nne',
                          'NORTHEAST' : 'ne',
                          'EAST-NORTHEAST': 'ene',
                          'EAST': 'e',
                          'EAST-SOUTHEAST': 'ese',
                          'SOUTHEAST': 'se',
                          'SOUTH-SOUTHEAST': 'sse',
                          'SOUTH': 's',
                          'SOUTH-SOUTHWEST': 'ssw',
                          'SOUTHWEST': 'sw',
                          'WEST-SOUTHWEST': 'wsw',
                          'WEST': 'w',
                          'WEST-NORTHWEST': 'wnw',
                          'NORTHWEST': 'nw',
                          'NORTH-NORTHWEST': 'nnw'}
        
        self.direction = dict_direction[direction]
        self.fws = fws_check

## Define hurricane input 

In [4]:
# define working directory
working_dir = '/Users/kyoung/Box Sync/qgis/hurricane/'

## First input advisory 

In [5]:
# Input hurricane data
hurr_id = 'al13'
hurr_yr = 2020
hurr_adv_type = '5day'
hurr_adv_num = 22

adv = Hurricane_Advisory(hurr_id, hurr_yr, hurr_adv_type, hurr_adv_num)
adv.get_advisory(working_dir)
gdf = adv.read_5dayForecast(working_dir)
cat = adv.get_max_wind(gdf)
adv.get_forwardspeed()
print(adv.get_storm())

Max Wind(kt): 100.0, Category: 3, key: [48.0], Max Wind Hour: ['27/0600']
17.261699999999998
wnw315i2.shp


## Second input advisory 

In [93]:
# Input hurricane data
hurr_id = 'al13'
hurr_yr = 2020
hurr_adv_type = '5day'
hurr_adv_num = 28

adv = Hurricane_Advisory(hurr_id, hurr_yr, hurr_adv_type, hurr_adv_num)
adv.get_advisory(working_dir)
gdf = adv.read_5dayForecast(working_dir)
cat = adv.get_max_wind(gdf)
adv.get_forwardspeed()
print(adv.get_storm())

Max Wind(kt): 130.0, Category: 4, key: [12.0], Max Wind Hour: ['27/0600']
14.960139999999999
nw415i2.shp
